<a href="https://colab.research.google.com/github/RoseSarlake/color-lab/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 28 10:44:37 2021

@author: Linna
"""
from CSL_homework_1 import read_envi
from CSL_homework_2 import *
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
from math import *
import os
os.chdir("ColorChecker_Specim_V10E\capture")

# ▪ Calculate reflectance spectral image Rsample(x,y,λ) 

# Raw spectral data of the ColorChecker chart (3-D data cube), 
# and wavelengths of the spectral bands:
sample, wavelengths = read_envi('colorchecker_sample_0007')

# Raw spectral data of the white reference sample (3-D data cube):
white = read_envi('WHITEREF_colorchecker_sample_0007')[0]

# Raw spectral data of the dark measurement (3-D data cube):
dark = read_envi('DARKREF_colorchecker_sample_0007')[0]

# Shapes of the data cubes:
shapeSample = sample.shape
shapeWhite  = white.shape
shapeDark   = dark.shape

# The number of rows for 'white' and 'dark' must 
# be the same as for the 'sample':

# White reference: Calculate mean along the row-direction:
white = white.mean(axis=0)

# Dimensions of 'white' should be now (1x1072x135), but Python 
# automatically removes the first singleton row-dimention. Let's 
# reshape 'white' into a 3-D data cube of shape (1x1072x135):
white = white.reshape((1,shapeWhite[1],shapeWhite[2]))

# Create a 3-D data cube of the same size as 'sample' by repeating 
# the mean row-slice along the row-direction 567 times:
white = white.repeat(shapeSample[0], axis=0)  

# Same steps are done for the 'dark' data cube 
# (in a more compact form):
dark = dark.mean(axis=0).reshape((1,shapeDark[1],shapeDark[2])).repeat(shapeSample[0],axis=0)


# Spectral reflectance image (3-D data cube):
refCube = (sample-dark)/(white-dark)

# ▪ Calculate RGB and grayscale images from the reflectance spectralimage
    # calculate the RGB image 
rgbImg = spim2rgb(refCube, wavelengths, lsource='D65')
plt.imshow(rgbImg)
    #calculate the grayscale iamge
grayImg = color.rgb2gray(rgbImg)
plt.imshow(grayImg)

# ▪ Calculate CIEL*a*b* color coordinates 
xyzCube = spim2XYZ(refCube, wavelengths, lsource='D65')
labCube = XYZ2Lab(xyzCube, cie_iluminant='D65')

# ▪ Implement and test ΔRGB, CIEΔE, and Michelson contrast calculation
# select 2 color patches first - eg. red and green
    # ΔRGB
rgb_red = (rgbImg[200,600,:])*256
rgb_green = (rgbImg[200,700,:])*256
delta_rgb = sqrt(np.square((rgb_red[0]-rgb_green[0]))+np.square((rgb_red[1]-rgb_green[1]))+np.square((rgb_red[2]-rgb_green[2])))
print(f"DELTA_RGB (red&green): {delta_rgb}")
    # CIEΔE
lab_red = labCube[200,600,:]
lab_green = labCube[200,700,:]
delta_E = sqrt(np.square((lab_red[0]-lab_green[0]))+np.square((lab_red[1]-lab_green[1]))+np.square((lab_red[2]-lab_green[2])))
print(f"DELTA_E (red&green): {delta_E}")
    #Michelson contrast
gray_red = grayImg[200,600]
gray_green = grayImg[200,700]
c = (max(gray_red,gray_green)-min(gray_red,gray_green))/(max(gray_red,gray_green)+min(gray_red,gray_green))
print(f"Michelson contrast (red&green): {c}")

# select another color
rgb_pink = (rgbImg[400,600,:])*256
lab_pink = labCube[400,600,:]
gray_pink = grayImg[400,600]
delta_rgb0 = sqrt(np.square((rgb_red[0]-rgb_pink[0]))+np.square((rgb_red[1]-rgb_pink[1]))+np.square((rgb_red[2]-rgb_pink[2])))
print(f"DELTA_RGB (red&pink): {delta_rgb0}")
delta_E0 = sqrt(np.square((lab_red[0]-lab_pink[0]))+np.square((lab_red[1]-lab_pink[1]))+np.square((lab_red[2]-lab_pink[2])))
print(f"DELTA_E (red&pink): {delta_E0}")
c0 = (max(gray_red,gray_pink)-min(gray_red,gray_pink))/(max(gray_red,gray_pink)+min(gray_red,gray_pink))
print(f"Michelson contrast (red&pink): {c0}")
